In [2]:
import re
import numpy as np
import pandas as pd 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, LSTM, Dropout
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.layers import MaxPooling1D

from matplotlib import pyplot
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer

porter=PorterStemmer()
stop_words = stopwords.words('english')

In [3]:
def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    tokens = word_tokenize(text)
#     tokens = [word for word in tokens if word not in stop_words]
    tokens = [porter.stem(word) for word in tokens]
    tokens = " ".join(tokens)
    return tokens